## Observations and Insights 

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata_df = pd.read_csv(mouse_metadata_path)
print(mouse_metadata_df)
#study_results_df = pd.read_csv(study_results_path)
#print(study_results_df)
# Combine the data into a single dataset


# Display the data table for preview


    Mouse ID Drug Regimen     Sex  Age_months  Weight (g)
0       k403     Ramicane    Male          21          16
1       s185    Capomulin  Female           3          17
2       x401    Capomulin  Female          16          15
3       m601    Capomulin    Male          22          17
4       g791     Ramicane    Male          11          16
..       ...          ...     ...         ...         ...
244     z314     Stelasyn  Female          21          28
245     z435     Propriva  Female          12          26
246     z581    Infubinol  Female          24          25
247     z795     Naftisol  Female          13          29
248     z969     Naftisol    Male           9          30

[249 rows x 5 columns]


In [24]:
# get an idea of the unique features in the mouse metadata
mouse_metadata_df.nunique()

Mouse ID        249
Drug Regimen     10
Sex               2
Age_months       24
Weight (g)       16
dtype: int64

In [26]:
# read and display study results seperately for clarity
study_results_df = pd.read_csv(study_results_path)
print(study_results_df)

     Mouse ID  Timepoint  Tumor Volume (mm3)  Metastatic Sites
0        b128          0           45.000000                 0
1        f932          0           45.000000                 0
2        g107          0           45.000000                 0
3        a457          0           45.000000                 0
4        c819          0           45.000000                 0
...       ...        ...                 ...               ...
1888     r944         45           41.581521                 2
1889     u364         45           31.023923                 3
1890     p438         45           61.433892                 1
1891     x773         45           58.634971                 4
1892     b879         45           72.555239                 2

[1893 rows x 4 columns]


In [25]:
# get an idea of the unique features in the study_results_df
study_results_df.nunique()

Mouse ID               249
Timepoint               10
Tumor Volume (mm3)    1644
Metastatic Sites         5
dtype: int64

In [37]:
# Combine the data into a single dataset
mouse_mdata_and_study_results_df = pd.merge(mouse_metadata_df,study_results_df)
mouse_mdata_and_study_results_df.columns
mouse_mdata_and_study_results_df.head(10)

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
5,k403,Ramicane,Male,21,16,25,33.464577,1
6,k403,Ramicane,Male,21,16,30,31.099498,1
7,k403,Ramicane,Male,21,16,35,26.546993,1
8,k403,Ramicane,Male,21,16,40,24.365505,1
9,k403,Ramicane,Male,21,16,45,22.050126,1


In [30]:
# Check the number of mice.
total_mice = mouse_mdata_and_study_results_df["Mouse ID"].count()
total_mice

1893

In [31]:
#get a view of mouse_mdata_and_study_results_df usinf .shape
mouse_mdata_and_study_results_df.shape

(1893, 8)

In [32]:
# Find any duplicate rows with the same Mouse ID's and Timepoints. 

mouse_mdata_and_study_results_df.duplicated(subset=['Mouse ID','Timepoint']).sum()

5

In [41]:
# Drop any duplicate rows
mouse_mdata_and_study_results_df.drop_duplicates(subset=['Mouse ID','Timepoint'],inplace=True)

In [42]:
# Recheck the number of mice
total_mice = mouse_mdata_and_study_results_df["Mouse ID"].count()
total_mice

1888

## Summary Statistics

In [48]:
# Generate a summary statistics table of mean, median, variance,
# standard deviation, and SEM of the tumor volume for each regimen.

#group and create analysis for each regiment and create a standard deviation, implimenting groupby method

drug_regimen_group = mouse_mdata_and_study_results_df.groupby(["Drug Regimen"])
drug_regimen_group
drug_regimen_mean = drug_regimen_group["Tumor Volume (mm3)"].mean()# looks for the tumor V mean
drug_regimen_median = drug_regimen_group["Tumor Volume (mm3)"].median()# looks for the tumor V median
drug_regiment_variance = drug_regimen_group["Tumor Volume (mm3)"].var()# looks for the tumor V variance
drug_regiment_std = drug_regimen_group["Tumor Volume (mm3)"].std()# looks for the tumor V standard deviation
drug_regiment_sem = drug_regimen_group["Tumor Volume (mm3)"].sem()# looks for the tumor V SEM
# to be continued 
stats_data = pd.DataFrame({"Mean":drug_regimen_mean,"Mean":drug_regimen_mean,"Mean":drug_regimen_mean,"Mean":drug_regimen_mean,"Mean":drug_regimen_mean,})
stats_data

## Bar and Pie Charts

In [13]:
# Generate a bar plot showing the total number of datapoints for each drug regimen using pandas.
# There should be a single bar per regimen



In [14]:
# General identical bar plot using pyplot instead of pandas.



In [15]:
# Generate a pie plot showing the distribution of female versus male mice using pandas



In [16]:
# Generate identical pie plot using pyplot



## Quartiles, Outliers and Boxplots

In [17]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# HINT: Not all mice lived until timepoint 45
# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


Calculate the quartiles and IQR and quantitatively determine if there are any potential outliers across all four treatment regimens.

In [18]:
# Calculate quartiles, IQR, and identify potential outliers for each regimen.

# One method to do this is the following, but you can use whatever method works for you.

##############################################################################
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds
    

In [19]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
# There should be a single chart with four box plots inside it.


## Line and Scatter Plots

In [20]:
# Generate a line plot of tumor volume vs. time point for a single mouse 
# treated with Capomulin


In [21]:
# Generate a scatter plot of average tumor volume vs. mouse weight 
# for all mice in the Capomulin regimen


## Correlation and Regression

In [22]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen


# Observations and Insights